In [1]:
import ee
import geemap
import numpy as np
import time

In [2]:
## authenticates the session with EE
ee.Authenticate()

# creates the connection with EE. 
ee.Initialize()

In [3]:
# Define a region of interest (ROI) for the analysis
roi = ee.Geometry.Rectangle([162.277817, -77.740157, 163.272100, -77.576571])  
# Coordinate range that covers the Taylor Valley lakes


In [4]:
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T2') \
            .filterBounds(roi)\
            .filterDate('2016-11-01', '2017-01-01')

In [5]:
# preprocessing for cloud masking
def maskL8sr(image):
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    qa = image.select('BQA')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
            .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [6]:
landsat8 = landsat8.map(maskL8sr)

print(landsat8.size().getInfo())

75


In [7]:
# define ROI's in specific images that will define lake ice/soil
roi_ice = ee.Geometry.Rectangle([163.077952, -77.616808, 163.077080, -77.616903])
roi_soil = ee.Geometry.Rectangle([163.078070, -77.625204, 163.075507, -77.625340])

In [8]:
# Define endmembers (example: using pseudo-invariant features)
# can add more endmembers in later, but need to select image and crop down to 
# specific regions in a single image
lake1 = ee.Image('LANDSAT/LC08/C02/T2/LC08_055116_20231205')
soil1 = ee.Image('LANDSAT/LC08/C02/T2/LC08_055116_20231205')

# clip original image down to the specific endmember spots
lakeice = lake1.clip(roi_ice)
soil = soil1.clip(roi_soil)

ice_mean = lakeice.reduceRegion(ee.Reducer.mean()).values()
soil_mean = soil.reduceRegion(ee.Reducer.mean()).values()

#endmember3 = ee.Image('YOUR_ENDMEMBER_IMAGE3')  # Replace with your endmember image

In [37]:
# Create an image with endmembers as bands
endmembers = ee.Image.cat([ice_mean, soil_mean])

AttributeError: 'Image' object has no attribute 'size'

In [38]:
# Function to apply SMA to an image
def apply_sma(image):
    return image.unmix(endmembers, True, True)


In [44]:
# Apply SMA to the image collection
fractional_abundances_col = landsat8.map(apply_sma)

In [40]:
# do a test SMA on one specific image, the one listed above: 
Image = ee.Image('LANDSAT/LC08/C02/T2/LC08_055116_20231205')
output = Image.unmix(endmembers, True, True)

In [42]:
# Export results (example: export first image)
def export_image(image, description, scale, region):
    task = ee.batch.Export.image.toDrive(
    image = image,
    description='fractions',
    folder = "SMA_EarthEngine",
    scale=30,
    region=roi
)
#task.start()
#print('Export task status:', task.status())

In [46]:
def export_image(image, description, scale, region):
  task = ee.batch.Export.image.toDrive(
      image=image,
      description=description,
      scale=scale,
      region=region
  )
  task.start()
  return task.id

def process_image(image):
  image_id = image.id()
  image_date = image.date().format('YYYYMMdd')
  description = f'fractional_abundances_{image_id}_{image_date}'
  task_id = export_image(image, description, 30, geometry)
  return ee.Feature(None, {'task_id': task_id})

def export_images(image_collection, description_prefix, scale, region):
  tasks = image_collection.map(process_image).flatten().collect()
  # ... (rest of your task monitoring code)

# Example usage
export_images(fractional_abundances_col, 'fractional_abundances', 30, roi)

EEException: A mapped function's arguments cannot be used in client-side operations

In [50]:
# allows to check for errors in google drive upload that don't show up as an
# outright error in the above code chunk. 
# Monitor tasks
def monitor_tasks():
    tasks = ee.batch.Task.list()
    for task in tasks:
        print(f'Task ID: {task.id}, Status: {task.status()}')

# Print status of all tasks
monitor_tasks()

print("Export task started. Check Google Drive for results.")

Task ID: WYSAOFUTM5FXJDVZFBB455I7, Status: {'state': 'FAILED', 'description': 'fraicks', 'priority': 100, 'creation_timestamp_ms': 1722391542799, 'update_timestamp_ms': 1722391551107, 'start_timestamp_ms': 1722391550516, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Collection.first: Error in map(ID=LC08_053116_20161101):\nImage.select: Band pattern 'BQA' did not match any bands. Available bands: [B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, QA_PIXEL, QA_RADSAT, SAA, SZA, VAA, VZA]", 'id': 'WYSAOFUTM5FXJDVZFBB455I7', 'name': 'projects/earthengine-legacy/operations/WYSAOFUTM5FXJDVZFBB455I7'}
Task ID: OKOTZ3OXVROIDV2DRLAM4KXN, Status: {'state': 'FAILED', 'description': 'fractional_abundances', 'priority': 100, 'creation_timestamp_ms': 1722391475898, 'update_timestamp_ms': 1722391485306, 'start_timestamp_ms': 1722391484772, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Collection.first: Error in map(ID=LC08_053116_20161101):\nImage.select: Band pattern 'BQA'

Task ID: 4ZNXS5SJSQO4A2BP65R4V64X, Status: {'state': 'FAILED', 'description': 'SMA of TV', 'priority': 100, 'creation_timestamp_ms': 1722368596350, 'update_timestamp_ms': 1722368602577, 'start_timestamp_ms': 1722368601326, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "ImageCollection.reduce: Error in map(ID=LC08_053116_20170104):\nImage.select: Can't add a band named 'B4' to image because a band with this name already exists. Existing bands: [B4, B3].", 'id': '4ZNXS5SJSQO4A2BP65R4V64X', 'name': 'projects/earthengine-legacy/operations/4ZNXS5SJSQO4A2BP65R4V64X'}
Task ID: JGFLWOV5CMZSWZZEJHCQWQ4O, Status: {'state': 'FAILED', 'description': 'SMA of TV', 'priority': 100, 'creation_timestamp_ms': 1722365701305, 'update_timestamp_ms': 1722365711847, 'start_timestamp_ms': 1722365711340, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "reduce.mean: Error in map(ID=LC08_053116_20170104):\nImage.select: Can't add a band named 'B4' to image because a band with this name 

Task ID: 4NIIEIN6DDGIZK52K22YNMFC, Status: {'state': 'FAILED', 'description': 'SMA of Taylor Valley with dummy endmembers', 'priority': 100, 'creation_timestamp_ms': 1722356453024, 'update_timestamp_ms': 1722356457996, 'start_timestamp_ms': 1722356457575, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Image.unmix, argument 'endmembers': Invalid type.\nExpected type: List<List<Float>>.\nActual type: Image<[array]>.", 'id': '4NIIEIN6DDGIZK52K22YNMFC', 'name': 'projects/earthengine-legacy/operations/4NIIEIN6DDGIZK52K22YNMFC'}
Task ID: BCAY2HYM2FMX7RIAJG5CNRAL, Status: {'state': 'FAILED', 'description': 'SMA of Taylor Valley with dummy endmembers', 'priority': 100, 'creation_timestamp_ms': 1722356432679, 'update_timestamp_ms': 1722356438626, 'start_timestamp_ms': 1722356437667, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Image.unmix, argument 'endmembers': Invalid type.\nExpected type: List<List<Float>>.\nActual type: Image<[array]>.", 'id': 'BCAY2HYM2FMX7RIA